In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, trim_messages # 함수 하나 추가

In [5]:
messages = [
    SystemMessage(content="System"), #너는 이런역할을 하는 ai다 라고 알려주는 부분
    HumanMessage(content="Human1"), #사용자가 이런 작업을 해줘 요청 하는 부분
    AIMessage(content="AI1"), #ai 가 응답한 내용을 적는 부분
    HumanMessage(content="Human2"), #사용자가 이런 작업을 해줘 요청 하는 부분
    AIMessage(content="AI2"), #ai 가 응답한 내용을 적는 부분
    HumanMessage(content="Human3"), #사용자가 이런 작업을 해줘 요청 하는 부분
    AIMessage(content="AI3"), #ai 가 응답한 내용을 적는 부분
]

In [6]:
# 대화 많이 남기기
trimmer = trim_messages(
    max_tokens=1000,  # 최대 토큰 수
    token_counter=llm
   )
trimmer.invoke(messages)

[SystemMessage(content='System', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human1', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI1', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI2', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [7]:
# 대화 적게 남기기
trimmer = trim_messages(
    max_tokens=30,  # 최대 토큰 수
    token_counter=llm
   )
trimmer.invoke(messages)

[HumanMessage(content='Human2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI2', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [9]:
# 과거 대화 남기고 최근 대화 지우기 
trimmer = trim_messages(
    max_tokens=30,  # 최대 토큰 수
    token_counter=llm,
    strategy = "first"
   )
trimmer.invoke(messages)

[SystemMessage(content='System', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human1', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI1', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human2', additional_kwargs={}, response_metadata={})]

In [10]:
# 최근 대화 남기고 과거 대화 지우기 
trimmer = trim_messages(
    max_tokens=30,  # 최대 토큰 수
    token_counter=llm,
    strategy = "last"
   )
trimmer.invoke(messages)

[HumanMessage(content='Human2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI2', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [ ]:
# 최근 대화 남기고 과거 대화 지우는데 시스템 메세지 남기기
trimmer = trim_messages(
    max_tokens=30,  # 최대 토큰 수
    token_counter=llm,
    strategy = "last",
    include_system=True  # 반드시 시스템 메시지는 남긴다.
   )
trimmer.invoke(messages)

[SystemMessage(content='System', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI2', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [13]:
# 최근 대화 남기고 과거 대화 지우는데 시스템 메세지 남기고 HumanMessage 부터 시작하기
trimmer = trim_messages(
    max_tokens=30,  # 최대 토큰 수
    token_counter=llm,
    strategy = "last",
    include_system=True,  # 반드시 시스템 메시지는 남긴다.
    start_on = "human"
   )
trimmer.invoke(messages)

[SystemMessage(content='System', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [14]:
messages = [
    SystemMessage(content="사용자의 질문에 2문장 이내로 짧게 대답해"), 
    HumanMessage(content="오늘은 피자를 먹어야지"), 
    AIMessage(content="정말 좋은 생각이야. 음료는 무엇으로 할거야?"), 
    HumanMessage(content="내일은 수영을 가야지!"),
    AIMessage(content="수영이라니, 정말 좋은 운동이야. 수영장은 어디로 다녀?"),
    HumanMessage(content="주말에는 영화를 보러 갈 거야!"), 
    AIMessage(content="주말이 벌써부터 기다려지겠는걸? 보려구 생각해둔 영화가 있어?"), 
]

In [15]:
trimmer = trim_messages(
    max_tokens=120,  # 최대 토큰 수
    token_counter=llm,
    strategy="last", # 최근대회만 남기기
    include_system=True,
    start_on="human"
)

In [16]:
trimmer.invoke(messages)

[SystemMessage(content='사용자의 질문에 2문장 이내로 짧게 대답해', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내일은 수영을 가야지!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='수영이라니, 정말 좋은 운동이야. 수영장은 어디로 다녀?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='주말에는 영화를 보러 갈 거야!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='주말이 벌써부터 기다려지겠는걸? 보려구 생각해둔 영화가 있어?', additional_kwargs={}, response_metadata={})]

In [17]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

chain = trimmer | llm | parser

In [21]:
chain.invoke(messages + [
    HumanMessage(content="오늘 뭘 먹는다고 했지?")
])

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableSequence > chain:trim_messages] Entering Chain run with input:
[inputs]
[chain/end] [chain:RunnableSequence > chain:trim_messages] s] Exiting Chain run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: 사용자의 질문에 2문장 이내로 짧게 대답해\nHuman: 주말에는 영화를 보러 갈 거야!\nAI: 주말이 벌써부터 기다려지겠는걸? 보려구 생각해둔 영화가 있어?\nHuman: 오늘 뭘 먹는다고 했지?"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [1.25s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
    

'오늘 먹을 음식에 대한 언급은 없었어, 뭘 먹을지 정했어?'

In [22]:
chain.invoke(messages + [
    HumanMessage(content="주말에 내가 뭐하러 간다고 했지?")
])

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableSequence > chain:trim_messages] Entering Chain run with input:
[inputs]
[chain/end] [chain:RunnableSequence > chain:trim_messages] s] Exiting Chain run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: 사용자의 질문에 2문장 이내로 짧게 대답해\nHuman: 주말에는 영화를 보러 갈 거야!\nAI: 주말이 벌써부터 기다려지겠는걸? 보려구 생각해둔 영화가 있어?\nHuman: 주말에 내가 뭐하러 간다고 했지?"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [1.20s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {

'주말에 영화를 보러 간다고 했어!'

In [20]:
import langchain
langchain.debug = True